In [6]:
import numpy as np
import esmvalcore.experimental as esmvaltool
import netCDF4 as nc
import warnings
import os
from ruamel.yaml import YAML
import yaml
import logging

warnings.filterwarnings("ignore", category=UserWarning, module='esmvalcore')


## Get HBVmountain recipe

In [7]:
recipe = esmvaltool.get_recipe('~/ESMValTool/esmvaltool/recipes/hydrology/recipe_newHBVmountain.yml')

## Functions

In [8]:
def select_cmip6_models(str_path_to_models, model_list=None, ignore_models=None, select_all=False):
    #Hier moet commentaar over wat deze functie in doet
    model_file = yaml.safe_load(open(str_path_to_models))
            
    if ignore_models != None:
        selected_CMIP6_models = dict((k, model_file[k]) for k in model_file
           if k not in ignore_models)
        selected_CMIP6_models = tuple(selected_CMIP6_models.values()) 
    if select_all == True and model_list == None:
        selected_CMIP6_models = dict((k, model_file[k]) for k in model_file.keys()
               if k in model_file)
        selected_CMIP6_models = tuple(selected_CMIP6_models.values()) 
    if model_list != None:
        selected_CMIP6_models = dict((k, model_file[k]) for k in model_list
               if k in model_file)
        selected_CMIP6_models = tuple(selected_CMIP6_models.values())
    return selected_CMIP6_models

In [9]:
def run_recipe_HBVmountain(recipe, catchment_name, scenario, CMIP6_models, start_year, end_year, catchment_id=None):
    recipe.data["preprocessors"]["daily"]["extract_shape"]["shapefile"] = os.path.join(catchment_name, catchment_name + '.' + 'shp') #shape
    recipe.data["diagnostics"]["diagnostic_daily"]["scripts"]["script"][
        "basin"
    ] = catchment_name #basin
    
    recipe.data["diagnostics"]["diagnostic_daily"]["additional_datasets"] = [
    *CMIP6_models
]
    variables = recipe.data["diagnostics"]["diagnostic_daily"]["variables"]
    var_names = "tas", "pr"
    
    for var_name in var_names:
        variables[var_name]["exp"] = scenario
        
        
    startyear = start_year #get_time(start_time).year
    for var_name in var_names:
        variables[var_name]["start_year"] = startyear

    endyear = end_year # get_time(end_time).year
    for var_name in var_names:
        variables[var_name]["end_year"] = endyear
    recipe_output = recipe.run()
    print(f'recipe stored in {os.path.dirname(str(list(recipe_output.values())[0].files[0].path))}')
    return recipe_output

## Select CMIP6 models

In [11]:
CMIP6_models = select_cmip6_models('climate_models.yml', select_all=True)


## Run recipe

In [12]:
recipe_output = run_recipe_HBVmountain(recipe, catchment_name='Conasauga', scenario='historical',
                                       CMIP6_models=CMIP6_models, start_year=1975, end_year=1978, catchment_id=None)